In [ ]:
## This file implements neural networks before and after lasso selection for p0017presabs_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p0017presabs_qual.csv')
df.shape

(253, 7157)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      2
1      0
2      2
3      2
4      2
      ..
248    2
249    1
250    0
251    2
252    2
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,TTTTTTTATAACATTCAAAGTCTCACCATTGTCATTTGAATGATCATCAATAATAATTAATTCGTAATCAGTACTCTTCATTGTTTGATTTAATACAGAA,...,group_1598,group_1744,group_4749,group_6727,group_8892,group_9205,group_9474,group_9475,group_9858,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
1,109,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,115,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,120335,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,2
4,120337,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2


In [6]:
df['pheno'].value_counts()

2    91
0    88
1    74
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 7156)

In [9]:
df_clean.head()

,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,TTTTTTTATAACATTCAAAGTCTCACCATTGTCATTTGAATGATCATCAATAATAATTAATTCGTAATCAGTACTCTTCATTGTTTGATTTAATACAGAA,TTTTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGA,...,group_1598,group_1744,group_4749,group_6727,group_8892,group_9205,group_9474,group_9475,group_9858,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,2
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 7156) (253,)


In [11]:
############# Fully-Connected Neural Network ################

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

Using TensorFlow backend.


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

In [14]:
dat = pd.DataFrame(X_test.iloc[:,0])
dat['test'] = y_test

In [15]:
dat

,id,test
112,NRS027,1
207,NRS246,1
186,NRS218,2
88,CFBRSa70,2
156,NRS177,1
...,...,...
244,SR3585,0
99,GA50819,2
147,NRS161,2
49,CFBREBSa114,1


In [16]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [17]:
#### neural network on over-sampling data
model1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [18]:
model1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model1.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 1ms/step - loss: 1.2350 - accuracy: 0.3672 - val_loss: 1.1179 - val_accuracy: 0.3553
Epoch 2/100
177/177 [==============================] - 0s 540us/step - loss: 1.0526 - accuracy: 0.4689 - val_loss: 1.0122 - val_accuracy: 0.4737
Epoch 3/100
177/177 [==============================] - 0s 554us/step - loss: 1.0044 - accuracy: 0.5141 - val_loss: 0.9859 - val_accuracy: 0.5395
Epoch 4/100
177/177 [==============================] - 0s 697us/step - loss: 0.9736 - accuracy: 0.5424 - val_loss: 0.9690 - val_accuracy: 0.5658
Epoch 5/100
177/177 [==============================] - 0s 609us/step - loss: 0.9297 - accuracy: 0.5763 - val_loss: 0.9857 - val_accuracy: 0.5526
Epoch 6/100
177/177 [==============================] - 0s 563us/step - loss: 0.8880 - accuracy: 0.6045 - val_loss: 0.9637 - val_accuracy: 0.5132
Epoch 7/100
177/177 [==============================] - 0s 461us/step - loss: 0.8694 - a

Epoch 57/100
177/177 [==============================] - 0s 682us/step - loss: 0.2680 - accuracy: 0.9040 - val_loss: 1.3290 - val_accuracy: 0.4474
Epoch 58/100
177/177 [==============================] - 0s 597us/step - loss: 0.2475 - accuracy: 0.9040 - val_loss: 1.2715 - val_accuracy: 0.4737
Epoch 59/100
177/177 [==============================] - 0s 696us/step - loss: 0.2265 - accuracy: 0.9435 - val_loss: 1.2210 - val_accuracy: 0.4605
Epoch 60/100
177/177 [==============================] - 0s 476us/step - loss: 0.2300 - accuracy: 0.9266 - val_loss: 1.2874 - val_accuracy: 0.4605
Epoch 61/100
177/177 [==============================] - 0s 432us/step - loss: 0.2266 - accuracy: 0.9322 - val_loss: 1.2809 - val_accuracy: 0.5132
Epoch 62/100
177/177 [==============================] - 0s 355us/step - loss: 0.2238 - accuracy: 0.9435 - val_loss: 1.2494 - val_accuracy: 0.4605
Epoch 63/100
177/177 [==============================] - 0s 413us/step - loss: 0.2404 - accuracy: 0.9096 - val_loss: 1.2892 -

In [52]:
acc_test1 = model1.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test1*100))

76/76 [==============================] - 0s 174us/step
test accuracy: 50.00%


In [20]:
pred = model1.predict_classes(X_test)
pred

array([0, 0, 2, 0, 1, 0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 1, 2, 0, 0, 0, 0,
       1, 1, 2, 0, 0, 1, 1, 2, 1, 2, 2, 1, 0, 2, 0, 2, 2, 0, 0, 0, 0, 1,
       2, 2, 1, 0, 2, 2, 2, 0, 0, 0, 1, 2, 1, 0, 2, 0, 1, 1, 0, 2, 1, 2,
       0, 2, 1, 1, 0, 0, 0, 1, 0, 2])

In [21]:
dat['pred'] = pred
dat

,id,test,pred
112,NRS027,1,0
207,NRS246,1,0
186,NRS218,2,2
88,CFBRSa70,2,0
156,NRS177,1,1
...,...,...,...
244,SR3585,0,0
99,GA50819,2,0
147,NRS161,2,1
49,CFBREBSa114,1,0


In [22]:
proba1 = model1.predict_proba(X_test)
dat_proba1 = pd.DataFrame(proba1)

In [23]:
dat_proba1

,0,1,2
0,0.897785,0.038647,0.063568
1,0.849721,0.078949,0.071330
2,0.003935,0.008331,0.987734
3,0.728429,0.046807,0.224764
4,0.011056,0.928673,0.060271
...,...,...,...
71,0.426559,0.176919,0.396522
72,0.984176,0.004903,0.010921
73,0.360253,0.636501,0.003246
74,0.997986,0.000290,0.001724


In [24]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [25]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p17p.csv", index = False,
         header=None)

In [56]:
hist1 = model1.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 467us/step - loss: 0.0872 - accuracy: 0.9605 - val_loss: 2.1276 - val_accuracy: 0.5395
Epoch 2/100
177/177 [==============================] - 0s 394us/step - loss: 0.0717 - accuracy: 0.9887 - val_loss: 2.0578 - val_accuracy: 0.5526
Epoch 3/100
177/177 [==============================] - 0s 367us/step - loss: 0.0914 - accuracy: 0.9718 - val_loss: 2.1070 - val_accuracy: 0.5132
Epoch 4/100
177/177 [==============================] - 0s 365us/step - loss: 0.0746 - accuracy: 0.9831 - val_loss: 2.1571 - val_accuracy: 0.4868
Epoch 5/100
177/177 [==============================] - 0s 392us/step - loss: 0.0857 - accuracy: 0.9661 - val_loss: 2.1716 - val_accuracy: 0.5132
Epoch 6/100
177/177 [==============================] - 0s 346us/step - loss: 0.0979 - accuracy: 0.9605 - val_loss: 2.1281 - val_accuracy: 0.5395
Epoch 7/100
177/177 [==============================] - 0s 364us/step - loss: 0.1263 -

Epoch 57/100
177/177 [==============================] - 0s 351us/step - loss: 0.0634 - accuracy: 0.9718 - val_loss: 2.5443 - val_accuracy: 0.5263
Epoch 58/100
177/177 [==============================] - 0s 654us/step - loss: 0.0587 - accuracy: 0.9774 - val_loss: 2.5746 - val_accuracy: 0.5000
Epoch 59/100
177/177 [==============================] - 0s 716us/step - loss: 0.0630 - accuracy: 0.9718 - val_loss: 2.5923 - val_accuracy: 0.5000
Epoch 60/100
177/177 [==============================] - 0s 704us/step - loss: 0.0552 - accuracy: 0.9774 - val_loss: 2.5751 - val_accuracy: 0.5000
Epoch 61/100
177/177 [==============================] - 0s 581us/step - loss: 0.0680 - accuracy: 0.9774 - val_loss: 2.5963 - val_accuracy: 0.4868
Epoch 62/100
177/177 [==============================] - 0s 725us/step - loss: 0.1026 - accuracy: 0.9548 - val_loss: 2.6196 - val_accuracy: 0.4737
Epoch 63/100
177/177 [==============================] - 0s 404us/step - loss: 0.0960 - accuracy: 0.9605 - val_loss: 2.6310 -

In [57]:
print('train accuracy: %.2f%%' % (np.mean(hist1.history['accuracy'])*100))

train accuracy: 96.82%


In [17]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [18]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS027,1,0,0.759813,0.001304,0.238883
1,p0017kpresabs_qual,NRS246,1,0,0.947819,0.009591,0.042591
2,p0017kpresabs_qual,NRS218,2,1,0.008693,0.989390,0.001916
3,p0017kpresabs_qual,CFBRSa70,2,0,0.813774,0.001482,0.184744
4,p0017kpresabs_qual,NRS177,1,1,0.000916,0.998926,0.000157
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS187,1,1,0.216843,0.568123,0.215034
604,p0040presabsSTCC_qual,SR1287,0,1,0.418802,0.553160,0.028038
605,p0040presabsSTCC_qual,CFBRSa50,0,0,0.948100,0.035031,0.016869
606,p0040presabsSTCC_qual,NRS196,2,2,0.000964,0.039095,0.959940


In [19]:
y_prob = df_proba[df_proba['phage']=='p0017presabs_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[8.97785100e-01, 3.86468000e-02, 6.35680300e-02],
       [8.49721100e-01, 7.89486300e-02, 7.13302940e-02],
       [3.93458040e-03, 8.33102900e-03, 9.87734400e-01],
       [7.28428800e-01, 4.68073000e-02, 2.24763910e-01],
       [1.10560880e-02, 9.28673150e-01, 6.02708270e-02],
       [9.81330100e-01, 1.86365640e-02, 3.33018580e-05],
       [9.44645000e-01, 6.40717600e-03, 4.89478200e-02],
       [9.91395300e-01, 5.01880400e-04, 8.10279900e-03],
       [8.46692860e-01, 1.15866980e-01, 3.74401730e-02],
       [7.69999600e-05, 1.09815836e-01, 8.90107200e-01],
       [7.31703800e-01, 1.78511430e-01, 8.97847640e-02],
       [5.19315600e-01, 2.03730730e-01, 2.76953670e-01],
       [5.20993700e-04, 1.60449580e-03, 9.97874500e-01],
       [1.79342220e-03, 2.59388720e-02, 9.72267700e-01],
       [5.59380700e-01, 3.26150800e-01, 1.14468470e-01],
       [9.45620600e-01, 5.31242530e-02, 1.25512150e-03],
       [1.44026980e-03, 9.93514200e-01, 5.04555550e-03],
       [1.55329510e-01, 5.10880

In [20]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [21]:
ovo1 = rocauc_ovo(y_test, y_prob, average="macro", multi_class="ovo")
ovo1

0.6214383059621155

In [22]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [23]:
ovr1 = rocauc_ovr(y_test, y_prob, average="macro", multi_class="ovr")
ovr1

0.6214383059621155

In [24]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y)

In [25]:
dat2 = pd.DataFrame(X_test.iloc[:,0])
dat2['test'] = y_test

In [26]:
dat2

,id,test
178,NRS210,2
104,Grady1,0
79,CFBRSa29,2
66,CFBRSa03,0
9,217,1
...,...,...
25,BCH-SA-10,1
94,GA15,2
244,SR3585,0
227,NRS387,2


In [27]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [28]:
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [31]:
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model2.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 1ms/step - loss: 1.1250 - accuracy: 0.3672 - val_loss: 1.0530 - val_accuracy: 0.4342
Epoch 2/100
177/177 [==============================] - 0s 640us/step - loss: 1.0484 - accuracy: 0.5367 - val_loss: 1.0734 - val_accuracy: 0.4605
Epoch 3/100
177/177 [==============================] - 0s 582us/step - loss: 1.0396 - accuracy: 0.5706 - val_loss: 1.1293 - val_accuracy: 0.5263
Epoch 4/100
177/177 [==============================] - 0s 668us/step - loss: 1.0606 - accuracy: 0.6102 - val_loss: 1.0775 - val_accuracy: 0.5263
Epoch 5/100
177/177 [==============================] - 0s 538us/step - loss: 0.8868 - accuracy: 0.6328 - val_loss: 1.0715 - val_accuracy: 0.5132
Epoch 6/100
177/177 [==============================] - 0s 549us/step - loss: 0.8385 - accuracy: 0.6158 - val_loss: 1.0881 - val_accuracy: 0.4211
Epoch 7/100
177/177 [==============================] - 0s 656us/step - loss: 0.8350 - a

Epoch 57/100
177/177 [==============================] - 0s 366us/step - loss: 0.2612 - accuracy: 0.9040 - val_loss: 1.7804 - val_accuracy: 0.4605
Epoch 58/100
177/177 [==============================] - 0s 414us/step - loss: 0.2319 - accuracy: 0.9153 - val_loss: 1.9355 - val_accuracy: 0.3947
Epoch 59/100
177/177 [==============================] - 0s 526us/step - loss: 0.2228 - accuracy: 0.9266 - val_loss: 1.8557 - val_accuracy: 0.5526
Epoch 60/100
177/177 [==============================] - 0s 476us/step - loss: 0.2634 - accuracy: 0.9209 - val_loss: 1.8534 - val_accuracy: 0.5132
Epoch 61/100
177/177 [==============================] - 0s 475us/step - loss: 0.2380 - accuracy: 0.9266 - val_loss: 2.0633 - val_accuracy: 0.4342
Epoch 62/100
177/177 [==============================] - 0s 593us/step - loss: 0.2341 - accuracy: 0.9040 - val_loss: 1.8609 - val_accuracy: 0.4605
Epoch 63/100
177/177 [==============================] - 0s 436us/step - loss: 0.2048 - accuracy: 0.9548 - val_loss: 1.9169 -

In [85]:
acc_test2 = model2.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

76/76 [==============================] - 0s 198us/step
test accuracy: 50.00%


In [33]:
pred2 = model2.predict_classes(X_test)
pred2

array([0, 1, 0, 1, 0, 2, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 1, 2, 0, 2, 2,
       0, 1, 1, 0, 0, 0, 2, 2, 1, 2, 2, 0, 2, 2, 2, 0, 1, 0, 2, 0, 2, 0,
       0, 0, 0, 2, 0, 2, 2, 1, 2, 1, 0, 0, 2, 0, 0, 2, 2, 2, 2, 2, 1, 1,
       0, 2, 2, 2, 0, 1, 1, 0, 2, 1])

In [34]:
dat2['pred'] = pred2
dat2

,id,test,pred
178,NRS210,2,0
104,Grady1,0,1
79,CFBRSa29,2,0
66,CFBRSa03,0,1
9,217,1,0
...,...,...,...
25,BCH-SA-10,1,1
94,GA15,2,1
244,SR3585,0,0
227,NRS387,2,2


In [35]:
proba2 = model2.predict_proba(X_test)
dat_proba2 = pd.DataFrame(proba2)

In [36]:
dat_proba2

,0,1,2
0,0.950409,0.048540,0.001051
1,0.136295,0.810181,0.053525
2,0.989155,0.005430,0.005414
3,0.192855,0.635855,0.171290
4,0.567618,0.301551,0.130831
...,...,...,...
71,0.000615,0.995850,0.003535
72,0.133443,0.770544,0.096013
73,0.740730,0.085304,0.173965
74,0.034299,0.004155,0.961546


In [37]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [38]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p17p.csv", index = False,
         header=None)

In [89]:
hist2 = model2.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 504us/step - loss: 0.1164 - accuracy: 0.9605 - val_loss: 2.6647 - val_accuracy: 0.4342
Epoch 2/100
177/177 [==============================] - 0s 410us/step - loss: 0.1091 - accuracy: 0.9492 - val_loss: 2.6360 - val_accuracy: 0.4737
Epoch 3/100
177/177 [==============================] - 0s 389us/step - loss: 0.1017 - accuracy: 0.9548 - val_loss: 2.6782 - val_accuracy: 0.4737
Epoch 4/100
177/177 [==============================] - 0s 357us/step - loss: 0.1029 - accuracy: 0.9435 - val_loss: 2.6764 - val_accuracy: 0.4342
Epoch 5/100
177/177 [==============================] - 0s 404us/step - loss: 0.0954 - accuracy: 0.9661 - val_loss: 2.7293 - val_accuracy: 0.4474
Epoch 6/100
177/177 [==============================] - 0s 390us/step - loss: 0.1169 - accuracy: 0.9492 - val_loss: 2.7168 - val_accuracy: 0.4868
Epoch 7/100
177/177 [==============================] - 0s 386us/step - loss: 0.1012 -

Epoch 57/100
177/177 [==============================] - 0s 396us/step - loss: 0.0964 - accuracy: 0.9435 - val_loss: 3.1253 - val_accuracy: 0.4474
Epoch 58/100
177/177 [==============================] - 0s 416us/step - loss: 0.0795 - accuracy: 0.9492 - val_loss: 3.0717 - val_accuracy: 0.5263
Epoch 59/100
177/177 [==============================] - 0s 353us/step - loss: 0.0667 - accuracy: 0.9718 - val_loss: 3.1390 - val_accuracy: 0.4474
Epoch 60/100
177/177 [==============================] - 0s 445us/step - loss: 0.0725 - accuracy: 0.9661 - val_loss: 3.1591 - val_accuracy: 0.4474
Epoch 61/100
177/177 [==============================] - 0s 452us/step - loss: 0.0630 - accuracy: 0.9661 - val_loss: 3.0914 - val_accuracy: 0.4211
Epoch 62/100
177/177 [==============================] - 0s 494us/step - loss: 0.0636 - accuracy: 0.9718 - val_loss: 3.0995 - val_accuracy: 0.4737
Epoch 63/100
177/177 [==============================] - 0s 586us/step - loss: 0.0591 - accuracy: 0.9831 - val_loss: 3.1670 -

In [90]:
print('train accuracy: %.2f%%' % (np.mean(hist2.history['accuracy'])*100))

train accuracy: 97.07%


In [29]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [30]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS210,2,0,0.999887,0.000112,8.851192e-07
1,p0017kpresabs_qual,Grady1,0,0,0.625329,0.369782,4.889404e-03
2,p0017kpresabs_qual,CFBRSa29,2,0,0.999098,0.000269,6.335156e-04
3,p0017kpresabs_qual,CFBRSa03,0,0,0.647338,0.331796,2.086646e-02
4,p0017kpresabs_qual,217,1,0,0.613342,0.381903,4.754707e-03
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS265,1,1,0.025601,0.687962,2.864372e-01
604,p0040presabsSTCC_qual,NY439,2,2,0.161947,0.266501,5.715521e-01
605,p0040presabsSTCC_qual,CFBRSa05,0,0,0.652983,0.254172,9.284494e-02
606,p0040presabsSTCC_qual,NRS205,2,2,0.000927,0.033760,9.653131e-01


In [31]:
y_prob2 = df_proba2[df_proba2['phage']=='p0017presabs_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[9.50409050e-01, 4.85404660e-02, 1.05052760e-03],
       [1.36294920e-01, 8.10180540e-01, 5.35245400e-02],
       [9.89155500e-01, 5.43018240e-03, 5.41440020e-03],
       [1.92855300e-01, 6.35854540e-01, 1.71290160e-01],
       [5.67618000e-01, 3.01551130e-01, 1.30830870e-01],
       [3.03387110e-05, 1.05399430e-04, 9.99864200e-01],
       [6.19640230e-01, 1.07306690e-01, 2.73053020e-01],
       [7.17105500e-03, 3.68505830e-02, 9.55978400e-01],
       [4.64002650e-07, 1.63578280e-05, 9.99983200e-01],
       [8.06335360e-02, 5.02502100e-03, 9.14341450e-01],
       [7.73711900e-01, 1.02803670e-01, 1.23484425e-01],
       [1.04009920e-02, 9.61994700e-03, 9.79979040e-01],
       [7.53824000e-01, 2.34950910e-01, 1.12250590e-02],
       [7.94750030e-01, 1.89111950e-01, 1.61380100e-02],
       [9.71428600e-01, 3.03582270e-03, 2.55356150e-02],
       [7.51547040e-01, 1.87612900e-01, 6.08401080e-02],
       [2.56840890e-05, 9.99973650e-01, 7.62742500e-07],
       [2.97899370e-01, 6.97555

In [32]:
ovo2 = rocauc_ovo(y_test, y_prob2, average="macro", multi_class="ovo")
ovo2

0.642089832566023

In [33]:
ovr2 = rocauc_ovr(y_test, y_prob2, average="macro", multi_class="ovr")
ovr2

0.642089832566023

In [34]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y)

In [35]:
dat3 = pd.DataFrame(X_test.iloc[:,0])
dat3['test'] = y_test

In [36]:
dat3

,id,test
231,NY360,2
92,EUH25,2
91,EUH15,2
203,NRS241,0
242,SR2852,2
...,...,...
26,BCH-SA-11,0
111,NRS022,1
96,GA27,2
129,NRS102,1


In [37]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [38]:
model3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [44]:
model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [45]:
model3.fit(X_train, y_train,
          batch_size=16, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 1ms/step - loss: 1.2870 - accuracy: 0.4068 - val_loss: 1.2340 - val_accuracy: 0.4211
Epoch 2/100
177/177 [==============================] - 0s 779us/step - loss: 1.0673 - accuracy: 0.5593 - val_loss: 1.3186 - val_accuracy: 0.4079
Epoch 3/100
177/177 [==============================] - 0s 762us/step - loss: 1.0118 - accuracy: 0.5763 - val_loss: 1.3027 - val_accuracy: 0.4211
Epoch 4/100
177/177 [==============================] - 0s 788us/step - loss: 0.9361 - accuracy: 0.6384 - val_loss: 1.3377 - val_accuracy: 0.3816
Epoch 5/100
177/177 [==============================] - 0s 644us/step - loss: 0.8932 - accuracy: 0.6158 - val_loss: 1.4061 - val_accuracy: 0.3947
Epoch 6/100
177/177 [==============================] - 0s 591us/step - loss: 0.8460 - accuracy: 0.6667 - val_loss: 1.3154 - val_accuracy: 0.4079
Epoch 7/100
177/177 [==============================] - 0s 944us/step - loss: 0.7988 - a

Epoch 57/100
177/177 [==============================] - 0s 589us/step - loss: 1.5373 - accuracy: 0.8362 - val_loss: 2.5570 - val_accuracy: 0.4474
Epoch 58/100
177/177 [==============================] - 0s 474us/step - loss: 1.2971 - accuracy: 0.8023 - val_loss: 2.2600 - val_accuracy: 0.4474
Epoch 59/100
177/177 [==============================] - 0s 472us/step - loss: 0.8201 - accuracy: 0.8644 - val_loss: 2.3750 - val_accuracy: 0.3684
Epoch 60/100
177/177 [==============================] - 0s 724us/step - loss: 0.5617 - accuracy: 0.8870 - val_loss: 1.9658 - val_accuracy: 0.4342
Epoch 61/100
177/177 [==============================] - 0s 591us/step - loss: 0.3823 - accuracy: 0.8983 - val_loss: 1.8755 - val_accuracy: 0.4211
Epoch 62/100
177/177 [==============================] - 0s 499us/step - loss: 0.2939 - accuracy: 0.9266 - val_loss: 1.8897 - val_accuracy: 0.4079
Epoch 63/100
177/177 [==============================] - 0s 609us/step - loss: 0.2697 - accuracy: 0.9096 - val_loss: 1.8971 -

In [122]:
acc_test3 = model3.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test3*100))

76/76 [==============================] - 0s 191us/step
test accuracy: 47.37%


In [46]:
pred3 = model3.predict_classes(X_test)
pred3

array([1, 0, 2, 1, 2, 2, 1, 2, 2, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 2, 0, 0, 2, 1, 0, 0, 2, 0, 1, 2, 1, 1, 2, 2, 0, 1, 1, 1, 0, 2,
       0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 0, 1, 1, 1, 1, 0, 1, 2, 0, 1, 1, 2,
       2, 0, 2, 1, 1, 1, 0, 2, 0, 0])

In [47]:
dat3['pred'] = pred3
dat3

,id,test,pred
231,NY360,2,1
92,EUH25,2,0
91,EUH15,2,2
203,NRS241,0,1
242,SR2852,2,2
...,...,...,...
26,BCH-SA-11,0,1
111,NRS022,1,0
96,GA27,2,2
129,NRS102,1,0


In [48]:
proba3 = model3.predict_proba(X_test)
dat_proba3 = pd.DataFrame(proba3)

In [49]:
dat_proba3

,0,1,2
0,0.300197,0.593951,1.058516e-01
1,0.999966,0.000034,2.497419e-13
2,0.148422,0.143526,7.080525e-01
3,0.020322,0.979353,3.254849e-04
4,0.000189,0.000286,9.995253e-01
...,...,...,...
71,0.152358,0.825816,2.182598e-02
72,0.554053,0.005847,4.401001e-01
73,0.000068,0.001090,9.988418e-01
74,0.960267,0.036225,3.508106e-03


In [50]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [51]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p17p.csv", index = False,
         header=None)

In [126]:
hist3 = model3.fit(X_train, y_train,
          batch_size=16, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 658us/step - loss: 0.1709 - accuracy: 0.9379 - val_loss: 1.8307 - val_accuracy: 0.4737
Epoch 2/100
177/177 [==============================] - 0s 539us/step - loss: 0.1685 - accuracy: 0.9548 - val_loss: 1.8430 - val_accuracy: 0.5132
Epoch 3/100
177/177 [==============================] - 0s 533us/step - loss: 0.2115 - accuracy: 0.9096 - val_loss: 1.7845 - val_accuracy: 0.5132
Epoch 4/100
177/177 [==============================] - 0s 526us/step - loss: 0.2423 - accuracy: 0.8927 - val_loss: 1.9230 - val_accuracy: 0.4474
Epoch 5/100
177/177 [==============================] - 0s 510us/step - loss: 0.2811 - accuracy: 0.9040 - val_loss: 1.8493 - val_accuracy: 0.4737
Epoch 6/100
177/177 [==============================] - 0s 527us/step - loss: 0.2603 - accuracy: 0.9209 - val_loss: 1.8055 - val_accuracy: 0.5000
Epoch 7/100
177/177 [==============================] - 0s 523us/step - loss: 0.2364 -

Epoch 57/100
177/177 [==============================] - 0s 952us/step - loss: 0.1407 - accuracy: 0.9322 - val_loss: 2.1878 - val_accuracy: 0.4211
Epoch 58/100
177/177 [==============================] - 0s 762us/step - loss: 0.1333 - accuracy: 0.9379 - val_loss: 2.2043 - val_accuracy: 0.4342
Epoch 59/100
177/177 [==============================] - 0s 678us/step - loss: 0.1335 - accuracy: 0.9492 - val_loss: 2.1604 - val_accuracy: 0.4474
Epoch 60/100
177/177 [==============================] - 0s 588us/step - loss: 0.1278 - accuracy: 0.9661 - val_loss: 2.1623 - val_accuracy: 0.4605
Epoch 61/100
177/177 [==============================] - 0s 663us/step - loss: 0.1230 - accuracy: 0.9718 - val_loss: 2.1666 - val_accuracy: 0.4474
Epoch 62/100
177/177 [==============================] - 0s 1ms/step - loss: 0.1537 - accuracy: 0.9492 - val_loss: 2.0941 - val_accuracy: 0.4474
Epoch 63/100
177/177 [==============================] - 0s 791us/step - loss: 0.1287 - accuracy: 0.9548 - val_loss: 2.2203 - v

In [127]:
print('train accuracy: %.2f%%' % (np.mean(hist3.history['accuracy'])*100))

train accuracy: 94.76%


In [39]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [40]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NY360,2,2,2.165526e-02,4.848140e-01,0.493531
1,p0017kpresabs_qual,EUH25,2,0,9.986388e-01,1.245148e-03,0.000116
2,p0017kpresabs_qual,EUH15,2,2,9.227520e-04,1.424882e-02,0.984828
3,p0017kpresabs_qual,NRS241,0,0,8.374333e-01,1.614128e-01,0.001154
4,p0017kpresabs_qual,SR2852,2,2,3.976981e-09,5.145955e-10,1.000000
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,BCH-SA-01,0,0,9.305407e-01,6.356251e-02,0.005897
604,p0040presabsSTCC_qual,504,0,1,4.476389e-02,8.577548e-01,0.097481
605,p0040presabsSTCC_qual,GA27,2,1,2.779456e-01,5.384378e-01,0.183617
606,p0040presabsSTCC_qual,NRS209,1,0,4.210180e-01,3.559393e-01,0.223043


In [41]:
y_prob3 = df_proba3[df_proba3['phage']=='p0017presabs_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[3.00197060e-01, 5.93951300e-01, 1.05851580e-01],
       [9.99965800e-01, 3.42531680e-05, 2.49741880e-13],
       [1.48421930e-01, 1.43525560e-01, 7.08052500e-01],
       [2.03219410e-02, 9.79352530e-01, 3.25484930e-04],
       [1.89209700e-04, 2.85522460e-04, 9.99525300e-01],
       [4.90020550e-02, 1.02699620e-01, 8.48298300e-01],
       [3.81876600e-02, 7.07397460e-01, 2.54414900e-01],
       [2.70405400e-03, 1.07550910e-03, 9.96220400e-01],
       [2.51256260e-01, 6.89154100e-02, 6.79828300e-01],
       [5.39077600e-01, 1.24853805e-01, 3.36068630e-01],
       [6.10260670e-01, 2.87556350e-01, 1.02182920e-01],
       [1.00000000e+00, 4.18953700e-08, 1.25978170e-14],
       [9.29220500e-01, 6.17651600e-02, 9.01423600e-03],
       [9.48855340e-01, 3.34729140e-02, 1.76718090e-02],
       [7.98825260e-01, 2.09601710e-03, 1.99078660e-01],
       [2.05057020e-01, 7.63409000e-01, 3.15339500e-02],
       [1.75715680e-01, 8.14795550e-01, 9.48872800e-03],
       [1.87083240e-01, 5.02405

In [42]:
ovo3 = rocauc_ovo(y_test, y_prob3, average="macro", multi_class="ovo")
ovo3

0.6114460935889507

In [43]:
ovr3 = rocauc_ovr(y_test, y_prob3, average="macro", multi_class="ovr")
ovr3

0.6114460935889507

In [44]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y)

In [45]:
dat4 = pd.DataFrame(X_test.iloc[:,0])
dat4['test'] = y_test

In [46]:
dat4

,id,test
236,SR1129,2
31,CA105,2
155,NRS175,1
92,EUH25,2
122,NRS070,2
...,...,...
235,SR1065,0
24,BCH-SA-09,0
104,Grady1,0
213,NRS254,2


In [47]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [48]:
model4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [57]:
model4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [58]:
model4.fit(X_train, y_train,
          batch_size=64, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 1ms/step - loss: 1.2188 - accuracy: 0.3333 - val_loss: 1.4709 - val_accuracy: 0.3947
Epoch 2/100
177/177 [==============================] - 0s 318us/step - loss: 1.0848 - accuracy: 0.5141 - val_loss: 1.1454 - val_accuracy: 0.4342
Epoch 3/100
177/177 [==============================] - 0s 422us/step - loss: 0.9350 - accuracy: 0.5819 - val_loss: 1.1533 - val_accuracy: 0.3684
Epoch 4/100
177/177 [==============================] - 0s 464us/step - loss: 0.8964 - accuracy: 0.5932 - val_loss: 1.2069 - val_accuracy: 0.3553
Epoch 5/100
177/177 [==============================] - 0s 493us/step - loss: 0.8521 - accuracy: 0.6158 - val_loss: 1.2739 - val_accuracy: 0.3289
Epoch 6/100
177/177 [==============================] - 0s 458us/step - loss: 0.8225 - accuracy: 0.6102 - val_loss: 1.3142 - val_accuracy: 0.3816
Epoch 7/100
177/177 [==============================] - 0s 463us/step - loss: 0.7911 - a

Epoch 57/100
177/177 [==============================] - 0s 472us/step - loss: 0.3007 - accuracy: 0.9209 - val_loss: 1.9965 - val_accuracy: 0.4211
Epoch 58/100
177/177 [==============================] - 0s 410us/step - loss: 0.2984 - accuracy: 0.9209 - val_loss: 2.0469 - val_accuracy: 0.4079
Epoch 59/100
177/177 [==============================] - 0s 334us/step - loss: 0.2983 - accuracy: 0.9153 - val_loss: 2.0762 - val_accuracy: 0.4211
Epoch 60/100
177/177 [==============================] - 0s 311us/step - loss: 0.2872 - accuracy: 0.9209 - val_loss: 2.0837 - val_accuracy: 0.4342
Epoch 61/100
177/177 [==============================] - 0s 379us/step - loss: 0.2829 - accuracy: 0.9322 - val_loss: 2.0854 - val_accuracy: 0.4079
Epoch 62/100
177/177 [==============================] - 0s 368us/step - loss: 0.2854 - accuracy: 0.9153 - val_loss: 2.0943 - val_accuracy: 0.3947
Epoch 63/100
177/177 [==============================] - 0s 342us/step - loss: 0.2843 - accuracy: 0.9040 - val_loss: 2.1305 -

In [163]:
acc_test4 = model4.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test4*100))

76/76 [==============================] - 0s 157us/step
test accuracy: 44.74%


In [59]:
pred4 = model4.predict_classes(X_test)
pred4

array([2, 2, 0, 1, 2, 2, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 2, 1, 1, 2,
       2, 0, 2, 2, 2, 0, 2, 2, 0, 2, 2, 0, 2, 1, 1, 1, 0, 0, 2, 0, 2, 0,
       1, 2, 0, 1, 1, 2, 2, 1, 0, 2, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 2, 2,
       2, 0, 2, 2, 0, 0, 2, 0, 2, 0])

In [60]:
dat4['pred'] = pred4
dat4

,id,test,pred
236,SR1129,2,2
31,CA105,2,2
155,NRS175,1,0
92,EUH25,2,1
122,NRS070,2,2
...,...,...,...
235,SR1065,0,0
24,BCH-SA-09,0,2
104,Grady1,0,0
213,NRS254,2,2


In [61]:
proba4 = model4.predict_proba(X_test)
dat_proba4 = pd.DataFrame(proba4)

In [62]:
dat_proba4

,0,1,2
0,0.024262,0.031831,0.943907
1,0.345262,0.032067,0.622672
2,0.913798,0.084018,0.002183
3,0.034562,0.919924,0.045514
4,0.010381,0.009723,0.979896
...,...,...,...
71,0.548255,0.000050,0.451695
72,0.057002,0.385987,0.557011
73,0.440886,0.434378,0.124736
74,0.042826,0.046443,0.910731


In [63]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [64]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p17p.csv", index = False,
         header=None)

In [167]:
hist4 = model4.fit(X_train, y_train,
          batch_size=64, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 439us/step - loss: 0.1800 - accuracy: 0.9379 - val_loss: 2.1268 - val_accuracy: 0.4474
Epoch 2/100
177/177 [==============================] - 0s 425us/step - loss: 0.1743 - accuracy: 0.9548 - val_loss: 2.1648 - val_accuracy: 0.4474
Epoch 3/100
177/177 [==============================] - 0s 355us/step - loss: 0.1796 - accuracy: 0.9492 - val_loss: 2.1813 - val_accuracy: 0.4474
Epoch 4/100
177/177 [==============================] - 0s 362us/step - loss: 0.1761 - accuracy: 0.9492 - val_loss: 2.1424 - val_accuracy: 0.4474
Epoch 5/100
177/177 [==============================] - 0s 383us/step - loss: 0.1707 - accuracy: 0.9492 - val_loss: 2.1463 - val_accuracy: 0.4605
Epoch 6/100
177/177 [==============================] - 0s 358us/step - loss: 0.1735 - accuracy: 0.9492 - val_loss: 2.1738 - val_accuracy: 0.4605
Epoch 7/100
177/177 [==============================] - 0s 372us/step - loss: 0.1722 -

Epoch 57/100
177/177 [==============================] - 0s 354us/step - loss: 0.1236 - accuracy: 0.9718 - val_loss: 2.4421 - val_accuracy: 0.4342
Epoch 58/100
177/177 [==============================] - 0s 380us/step - loss: 0.1260 - accuracy: 0.9718 - val_loss: 2.4330 - val_accuracy: 0.4342
Epoch 59/100
177/177 [==============================] - 0s 394us/step - loss: 0.1183 - accuracy: 0.9661 - val_loss: 2.4426 - val_accuracy: 0.4211
Epoch 60/100
177/177 [==============================] - 0s 377us/step - loss: 0.1144 - accuracy: 0.9605 - val_loss: 2.4756 - val_accuracy: 0.4211
Epoch 61/100
177/177 [==============================] - 0s 373us/step - loss: 0.1144 - accuracy: 0.9661 - val_loss: 2.4587 - val_accuracy: 0.4211
Epoch 62/100
177/177 [==============================] - 0s 344us/step - loss: 0.1134 - accuracy: 0.9661 - val_loss: 2.4452 - val_accuracy: 0.4474
Epoch 63/100
177/177 [==============================] - 0s 328us/step - loss: 0.1137 - accuracy: 0.9718 - val_loss: 2.4589 -

In [168]:
print('train accuracy: %.2f%%' % (np.mean(hist4.history['accuracy'])*100))

train accuracy: 96.31%


In [49]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [50]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,SR1129,2,2,4.821690e-07,0.000001,9.999983e-01
1,p0017kpresabs_qual,CA105,2,2,2.652370e-03,0.000049,9.972990e-01
2,p0017kpresabs_qual,NRS175,1,1,1.198157e-02,0.988018,5.232074e-09
3,p0017kpresabs_qual,EUH25,2,1,1.388957e-01,0.735954,1.251503e-01
4,p0017kpresabs_qual,NRS070,2,1,7.231966e-03,0.810317,1.824512e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS247,1,2,2.957360e-02,0.385730,5.846961e-01
604,p0040presabsSTCC_qual,NRS215,1,2,3.196635e-03,0.445057,5.517461e-01
605,p0040presabsSTCC_qual,SR4152,2,2,4.319404e-02,0.028984,9.278219e-01
606,p0040presabsSTCC_qual,NRS035,1,1,4.902312e-03,0.680427,3.146706e-01


In [51]:
y_prob4 = df_proba4[df_proba4['phage']=='p0017presabs_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[2.42624990e-02, 3.18308360e-02, 9.43906660e-01],
       [3.45261500e-01, 3.20665700e-02, 6.22671840e-01],
       [9.13798400e-01, 8.40183500e-02, 2.18328300e-03],
       [3.45619100e-02, 9.19923700e-01, 4.55143640e-02],
       [1.03811610e-02, 9.72315500e-03, 9.79895700e-01],
       [6.84882370e-03, 8.28253600e-02, 9.10325800e-01],
       [1.53362200e-01, 7.86886750e-01, 5.97510800e-02],
       [8.80036500e-01, 5.11755840e-03, 1.14846006e-01],
       [1.57277000e-01, 6.39489200e-01, 2.03233760e-01],
       [3.35859840e-13, 9.99999640e-01, 3.50026970e-07],
       [1.08816310e-01, 8.69485500e-01, 2.16982250e-02],
       [1.64301760e-05, 9.99952200e-01, 3.12962800e-05],
       [4.94104860e-01, 2.19399100e-01, 2.86496040e-01],
       [1.66820120e-01, 5.34168400e-01, 2.99011440e-01],
       [8.74798660e-01, 1.23340200e-01, 1.86113760e-03],
       [1.54968590e-01, 7.07158600e-01, 1.37872740e-01],
       [5.14315500e-06, 9.99715030e-01, 2.79777330e-04],
       [5.62075300e-01, 8.09830

In [52]:
ovo4 = rocauc_ovo(y_test, y_prob4, average="macro", multi_class="ovo")
ovo4

0.6109507776174444

In [53]:
ovr4 = rocauc_ovr(y_test, y_prob4, average="macro", multi_class="ovr")
ovr4

0.6109507776174444

In [54]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.6214812524336334

In [55]:
np.std(ovos)

0.012612600171702732

In [56]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.6214812524336334

In [57]:
np.std(ovrs)

0.012612600171702732

In [169]:
accs = [acc_test1, acc_test2, acc_test3, acc_test4]

In [170]:
mean = np.mean(accs)
print('test accuracy mean: %.2f%%' % (mean*100))

test accuracy mean: 48.03%


In [171]:
std = np.std(accs)
print('test accuracy standard deviation:', std)

test accuracy standard deviation: 0.021819902064750783


In [172]:
accs_train = [np.mean(hist1.history['accuracy']), np.mean(hist2.history['accuracy']), np.mean(hist3.history['accuracy']),
             np.mean(hist4.history['accuracy'])]

In [173]:
mean_train = np.mean(accs_train)
print('train accuracy mean: %.2f%%' % (mean_train*100))

train accuracy mean: 96.24%


In [174]:
std_train = np.std(accs_train)
print('train accuracy standard deviation:', std_train)

train accuracy standard deviation: 0.009000052


In [82]:
############ Feature selection using lasso ##########

In [58]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [59]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
selection.fit(X.loc[:, X.columns != 'id'], y)

SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None,
                                             solver='liblinear', tol=0.0001,
                                             verbose=0, warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [60]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [61]:
X_train_features = np.vstack((names, X.loc[:, X.columns != 'id']))
X_train_features = pd.DataFrame(X_train_features)

In [62]:
sel_features = X_train_features.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 7155
selected features: 319


In [63]:
cols = sel_features.values
cols.reshape((1, -1))

array([[   9,   11,   38,   40,  107,  130,  211,  225,  227,  235,  267,
         270,  297,  310,  313,  345,  362,  391,  422,  459,  490,  498,
         527,  533,  540,  551,  626,  690,  745,  760,  785,  805,  881,
         899,  975,  982,  983, 1047, 1064, 1066, 1114, 1135, 1162, 1181,
        1201, 1202, 1208, 1221, 1255, 1278, 1287, 1288, 1309, 1321, 1322,
        1328, 1391, 1400, 1403, 1407, 1414, 1427, 1442, 1461, 1481, 1484,
        1501, 1519, 1545, 1598, 1600, 1619, 1650, 1653, 1654, 1700, 1738,
        1739, 1746, 1801, 1803, 1815, 1830, 1851, 1853, 1859, 1885, 1921,
        1946, 1995, 2014, 2098, 2138, 2165, 2171, 2193, 2200, 2218, 2249,
        2258, 2303, 2311, 2348, 2350, 2372, 2393, 2418, 2461, 2481, 2492,
        2555, 2563, 2572, 2612, 2622, 2625, 2627, 2709, 2732, 2734, 2738,
        2746, 2754, 2794, 2805, 2851, 2895, 2898, 2934, 2962, 3010, 3022,
        3053, 3072, 3083, 3114, 3131, 3150, 3184, 3209, 3221, 3230, 3266,
        3311, 3372, 3409, 3426, 3513, 

In [64]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGA',
       'TTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAAAGTGCTTGAAAGATCATAGATCATCACGTTCAAGTCAATTTTAGCCTCGGCAGGAT',
       'TTTTTTAGTTCCTACGAATAAGACTTGTCCACCATCTTCTGAAACTTGTTTCAAGAAGTTGTATGCCTCGTCTACTTTTTTCACTGTTTTTTGTAAGTCG',
       'TTTTTTAGCTTCAATGATAGCAGACATTCAGACACCTCCATTTAAATTTTTGGTGCTTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAA',
       'TTTTTAGTTCCTACGAATAAGACTTGTCCACCATCTTCTGAAACTTGTTTCAAGAAGTTGTATGCCTCGTCTACTTTTTTCACTGTTTTTTGTAAGTCGA',
       'TTTTGTTGGAGTA', 'TTTTCATTTTCATTT',
       'TTTTCAATTATAGCACTTCCATTATGATAATAGGTTTGTCTAAGATTGCTTAAAGCAGTTCCCTGCAATTCAGACTTTTTTTGTAAATCTTTTCCATTTA',
       'TTTTCAATAAAATCTAAAACATCGCCAACTTTATCGCCAAGCCATTTGGTACCTTTACCTATTTGATCTTTTGTCCAGTTAAATGCCGATGATGCACCGG',
       'TTTTATTGTATAAT',
       'TTTTAGTTCCTACGAATAAGACTTGTCCACCATCTTCTGAAACTTGTTTCAAGAAGTTGTATGCCTCGTCTACTTTTTTCACTGTTTTTTGTAAGTCGAT',
       'TTTTAGGTAATGAATACGAAAATTTCGTATTTACGA

In [65]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']

In [66]:
df_sel['strain'] = X.iloc[:,0]

In [67]:
df_sel

,TTTTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGA,TTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAAAGTGCTTGAAAGATCATAGATCATCACGTTCAAGTCAATTTTAGCCTCGGCAGGAT,TTTTTTAGTTCCTACGAATAAGACTTGTCCACCATCTTCTGAAACTTGTTTCAAGAAGTTGTATGCCTCGTCTACTTTTTTCACTGTTTTTTGTAAGTCG,TTTTTTAGCTTCAATGATAGCAGACATTCAGACACCTCCATTTAAATTTTTGGTGCTTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAA,TTTTTAGTTCCTACGAATAAGACTTGTCCACCATCTTCTGAAACTTGTTTCAAGAAGTTGTATGCCTCGTCTACTTTTTTCACTGTTTTTTGTAAGTCGA,TTTTGTTGGAGTA,TTTTCATTTTCATTT,TTTTCAATTATAGCACTTCCATTATGATAATAGGTTTGTCTAAGATTGCTTAAAGCAGTTCCCTGCAATTCAGACTTTTTTTGTAAATCTTTTCCATTTA,TTTTCAATAAAATCTAAAACATCGCCAACTTTATCGCCAAGCCATTTGGTACCTTTACCTATTTGATCTTTTGTCCAGTTAAATGCCGATGATGCACCGG,TTTTATTGTATAAT,...,group_2498,group_7793,group_4054,group_276,group_178,group_64,group_1598,group_8892,pheno,strain
0,1,0,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,2,107
1,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,109
2,0,0,1,0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,2,115
3,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,120335
4,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,120337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0,0,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,2,SR4152
249,0,0,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,1,SR4153
250,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,SR4155
251,0,0,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,2,SR4156


In [68]:
X_sel = df_sel.loc[:, df_sel.columns != 'pheno']
y_sel = df_sel['pheno']
print(X_sel.shape, y_sel.shape, df_sel.shape)

(253, 320) (253,) (253, 321)


In [69]:
df_sel['pheno'].value_counts()

2    91
0    88
1    74
Name: pheno, dtype: int64

In [70]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_sel)

In [71]:
dat5 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat5['test'] = y_sel_test

In [72]:
dat5

,strain,test
231,NY360,2
139,NRS113,0
4,120337,2
59,CFBREBSa127,1
96,GA27,2
...,...,...
236,SR1129,2
145,NRS152,2
86,CFBRSa66A,0
28,BCH-SA-13,1


In [73]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [81]:
#### neural network on over-sampling data
model_sel = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [82]:
model_sel.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [83]:
model_sel.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 867us/step - loss: 1.0777 - accuracy: 0.3842 - val_loss: 1.2104 - val_accuracy: 0.4079
Epoch 2/100
177/177 [==============================] - 0s 301us/step - loss: 0.9825 - accuracy: 0.5085 - val_loss: 1.2137 - val_accuracy: 0.4605
Epoch 3/100
177/177 [==============================] - 0s 240us/step - loss: 0.9385 - accuracy: 0.5254 - val_loss: 1.1150 - val_accuracy: 0.5000
Epoch 4/100
177/177 [==============================] - 0s 227us/step - loss: 0.8887 - accuracy: 0.5876 - val_loss: 1.1137 - val_accuracy: 0.5132
Epoch 5/100
177/177 [==============================] - 0s 251us/step - loss: 0.8661 - accuracy: 0.5819 - val_loss: 1.1015 - val_accuracy: 0.5132
Epoch 6/100
177/177 [==============================] - 0s 282us/step - loss: 0.8406 - accuracy: 0.5989 - val_loss: 1.0997 - val_accuracy: 0.5000
Epoch 7/100
177/177 [==============================] - 0s 216us/step - loss: 0.8694 -

In [202]:
acc_test_sel = model_sel.evaluate(X_sel_test, y_sel_test)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_sel*100))

76/76 [==============================] - 0s 64us/step
over-sampling test accuracy: 61.84%


In [84]:
pred5 = model_sel.predict_classes(X_sel_test)
pred5

array([0, 0, 2, 2, 2, 1, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 2, 1, 1,
       2, 0, 0, 2, 0, 1, 0, 1, 1, 2, 1, 0, 1, 1, 0, 1, 2, 0, 1, 0, 0, 2,
       2, 1, 2, 0, 2, 2, 2, 2, 0, 2, 0, 0, 1, 0, 2, 1, 0, 0, 0, 1, 1, 2,
       1, 2, 0, 0, 0, 2, 2, 0, 2, 2])

In [85]:
dat5['pred'] = pred5
dat5

,strain,test,pred
231,NY360,2,0
139,NRS113,0,0
4,120337,2,2
59,CFBREBSa127,1,2
96,GA27,2,2
...,...,...,...
236,SR1129,2,2
145,NRS152,2,2
86,CFBRSa66A,0,0
28,BCH-SA-13,1,2


In [86]:
proba5 = model_sel.predict_proba(X_sel_test)
dat_proba5 = pd.DataFrame(proba5)

In [87]:
dat_proba5

,0,1,2
0,0.547793,0.451841,0.000366
1,0.964647,0.025468,0.009885
2,0.011376,0.003404,0.985219
3,0.069790,0.236063,0.694147
4,0.000054,0.006340,0.993606
...,...,...,...
71,0.027655,0.141378,0.830968
72,0.003434,0.010073,0.986493
73,0.936168,0.048268,0.015564
74,0.005631,0.110510,0.883859


In [88]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [89]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p17p.csv", index = False,
         header=None)

In [206]:
hist_sel = model_sel.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 195us/step - loss: 0.2121 - accuracy: 0.9492 - val_loss: 1.3316 - val_accuracy: 0.5921
Epoch 2/100
177/177 [==============================] - 0s 226us/step - loss: 0.2564 - accuracy: 0.9096 - val_loss: 1.4493 - val_accuracy: 0.5658
Epoch 3/100
177/177 [==============================] - 0s 232us/step - loss: 0.1995 - accuracy: 0.9548 - val_loss: 1.3716 - val_accuracy: 0.5658
Epoch 4/100
177/177 [==============================] - 0s 190us/step - loss: 0.1865 - accuracy: 0.9492 - val_loss: 1.3109 - val_accuracy: 0.5658
Epoch 5/100
177/177 [==============================] - 0s 163us/step - loss: 0.1802 - accuracy: 0.9492 - val_loss: 1.3788 - val_accuracy: 0.5921
Epoch 6/100
177/177 [==============================] - 0s 166us/step - loss: 0.1710 - accuracy: 0.9548 - val_loss: 1.4030 - val_accuracy: 0.5921
Epoch 7/100
177/177 [==============================] - 0s 174us/step - loss: 0.1812 -

Epoch 57/100
177/177 [==============================] - 0s 130us/step - loss: 0.1101 - accuracy: 0.9831 - val_loss: 1.5576 - val_accuracy: 0.6184
Epoch 58/100
177/177 [==============================] - 0s 256us/step - loss: 0.1141 - accuracy: 0.9605 - val_loss: 1.6472 - val_accuracy: 0.6053
Epoch 59/100
177/177 [==============================] - 0s 192us/step - loss: 0.4902 - accuracy: 0.8983 - val_loss: 2.1112 - val_accuracy: 0.5789
Epoch 60/100
177/177 [==============================] - 0s 202us/step - loss: 0.4876 - accuracy: 0.9492 - val_loss: 1.8792 - val_accuracy: 0.6184
Epoch 61/100
177/177 [==============================] - 0s 189us/step - loss: 0.3805 - accuracy: 0.9548 - val_loss: 1.7504 - val_accuracy: 0.5921
Epoch 62/100
177/177 [==============================] - 0s 213us/step - loss: 0.2678 - accuracy: 0.9605 - val_loss: 1.6064 - val_accuracy: 0.6053
Epoch 63/100
177/177 [==============================] - 0s 208us/step - loss: 0.1535 - accuracy: 0.9605 - val_loss: 1.4836 -

In [207]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel.history['accuracy'])*100))

train accuracy: 96.45%


In [74]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [75]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NY360,2,1,3.848032e-03,0.996096,0.000056
1,p0017kpresabs_qual,NRS113,0,0,8.441349e-01,0.000015,0.155850
2,p0017kpresabs_qual,120337,2,2,1.466081e-02,0.005713,0.979626
3,p0017kpresabs_qual,CFBREBSa127,1,0,9.985392e-01,0.000350,0.001111
4,p0017kpresabs_qual,GA27,2,2,1.665667e-08,0.000005,0.999995
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,CFBREBSa131,2,1,6.617341e-03,0.565843,0.427540
604,p0040presabsSTCC_qual,NRS112,2,2,1.055758e-03,0.045611,0.953333
605,p0040presabsSTCC_qual,BCH-SA-06,0,0,8.747076e-01,0.071162,0.054130
606,p0040presabsSTCC_qual,834N,2,2,2.593112e-06,0.000013,0.999985


In [76]:
y_prob5 = df_proba5[df_proba5['phage']=='p0017presabs_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[5.47793270e-01, 4.51840800e-01, 3.65946270e-04],
       [9.64646700e-01, 2.54682450e-02, 9.88508200e-03],
       [1.13764815e-02, 3.40412560e-03, 9.85219360e-01],
       [6.97899460e-02, 2.36062910e-01, 6.94147170e-01],
       [5.38612150e-05, 6.34027400e-03, 9.93605850e-01],
       [1.91742500e-01, 6.82658140e-01, 1.25599460e-01],
       [2.53399200e-01, 3.08578130e-01, 4.38022670e-01],
       [6.27517550e-02, 9.31558430e-01, 5.68989340e-03],
       [4.05248720e-05, 7.15595050e-08, 9.99959350e-01],
       [9.60366370e-01, 3.89166000e-02, 7.17021350e-04],
       [8.36825800e-01, 1.57177480e-01, 5.99673500e-03],
       [2.02666860e-01, 2.36815260e-04, 7.97096300e-01],
       [2.18478680e-01, 5.00190500e-01, 2.81330760e-01],
       [9.92467800e-01, 5.59619700e-03, 1.93596030e-03],
       [1.22960415e-02, 8.44317100e-03, 9.79260740e-01],
       [7.93340060e-02, 9.20149270e-01, 5.16795030e-04],
       [3.21384430e-01, 5.03609900e-01, 1.75005690e-01],
       [9.80847200e-01, 1.45017

In [77]:
ovo5 = rocauc_ovo(y_sel_test, y_prob5, average="macro", multi_class="ovo")
ovo5

0.7808727868251677

In [78]:
ovr5 = rocauc_ovr(y_sel_test, y_prob5, average="macro", multi_class="ovr")
ovr5

0.7808727868251677

In [79]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_sel)

In [80]:
dat6 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat6['test'] = y_sel_test

In [81]:
dat6

,strain,test
44,CFBREBSa104,1
170,NRS199,2
197,NRS233,1
238,SR1746,0
172,NRS202,2
...,...,...
92,EUH25,2
217,NRS260,0
75,CFBRSa25,0
156,NRS177,1


In [82]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [83]:
model_sel2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [95]:
model_sel2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [96]:
model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 658us/step - loss: 1.1542 - accuracy: 0.4011 - val_loss: 1.0861 - val_accuracy: 0.4342
Epoch 2/100
177/177 [==============================] - 0s 72us/step - loss: 1.0443 - accuracy: 0.4633 - val_loss: 1.0538 - val_accuracy: 0.4605
Epoch 3/100
177/177 [==============================] - 0s 99us/step - loss: 1.0133 - accuracy: 0.5367 - val_loss: 1.0260 - val_accuracy: 0.4737
Epoch 4/100
177/177 [==============================] - 0s 103us/step - loss: 0.9923 - accuracy: 0.5424 - val_loss: 0.9969 - val_accuracy: 0.4868
Epoch 5/100
177/177 [==============================] - 0s 98us/step - loss: 0.9630 - accuracy: 0.5537 - val_loss: 0.9733 - val_accuracy: 0.4868
Epoch 6/100
177/177 [==============================] - 0s 123us/step - loss: 0.9397 - accuracy: 0.5876 - val_loss: 0.9607 - val_accuracy: 0.5132
Epoch 7/100
177/177 [==============================] - 0s 106us/step - loss: 0.9182 - ac

177/177 [==============================] - 0s 92us/step - loss: 0.4023 - accuracy: 0.8983 - val_loss: 0.8803 - val_accuracy: 0.5921
Epoch 58/100
177/177 [==============================] - 0s 96us/step - loss: 0.3984 - accuracy: 0.9040 - val_loss: 0.8762 - val_accuracy: 0.6184
Epoch 59/100
177/177 [==============================] - 0s 79us/step - loss: 0.3929 - accuracy: 0.9153 - val_loss: 0.8826 - val_accuracy: 0.5921
Epoch 60/100
177/177 [==============================] - 0s 98us/step - loss: 0.3881 - accuracy: 0.9209 - val_loss: 0.8918 - val_accuracy: 0.5789
Epoch 61/100
177/177 [==============================] - 0s 69us/step - loss: 0.3789 - accuracy: 0.9096 - val_loss: 0.8887 - val_accuracy: 0.5789
Epoch 62/100
177/177 [==============================] - 0s 86us/step - loss: 0.3749 - accuracy: 0.9153 - val_loss: 0.8781 - val_accuracy: 0.6316
Epoch 63/100
177/177 [==============================] - 0s 84us/step - loss: 0.3737 - accuracy: 0.9209 - val_loss: 0.8753 - val_accuracy: 0.605

In [235]:
acc_test_sel2 = model_sel2.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel2*100))

76/76 [==============================] - 0s 142us/step
test accuracy: 63.16%


In [97]:
pred6 = model_sel2.predict_classes(X_sel_test)
pred6

array([2, 2, 0, 2, 2, 2, 2, 0, 1, 0, 1, 1, 0, 2, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0, 2, 1, 2, 0, 1, 2, 0, 1, 2, 1, 2,
       1, 1, 1, 1, 0, 1, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 1, 2, 2, 0, 2,
       0, 0, 0, 2, 2, 0, 0, 0, 1, 2])

In [98]:
dat6['pred'] = pred6
dat6

,strain,test,pred
44,CFBREBSa104,1,2
170,NRS199,2,2
197,NRS233,1,0
238,SR1746,0,2
172,NRS202,2,2
...,...,...,...
92,EUH25,2,0
217,NRS260,0,0
75,CFBRSa25,0,0
156,NRS177,1,1


In [99]:
proba6 = model_sel2.predict_proba(X_sel_test)
dat_proba6 = pd.DataFrame(proba6)

In [100]:
dat_proba6

,0,1,2
0,0.222783,0.039238,0.737979
1,0.190075,0.206699,0.603226
2,0.500600,0.206930,0.292469
3,0.350031,0.016853,0.633117
4,0.009613,0.043573,0.946814
...,...,...,...
71,0.837619,0.137569,0.024812
72,0.980481,0.016112,0.003407
73,0.613046,0.172551,0.214402
74,0.008814,0.985286,0.005900


In [101]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [102]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p17p.csv", index = False,
         header=None)

In [239]:
hist_sel2 = model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 78us/step - loss: 0.2492 - accuracy: 0.9266 - val_loss: 0.8825 - val_accuracy: 0.6053
Epoch 2/100
177/177 [==============================] - 0s 83us/step - loss: 0.2445 - accuracy: 0.9379 - val_loss: 0.8834 - val_accuracy: 0.6184
Epoch 3/100
177/177 [==============================] - 0s 81us/step - loss: 0.2415 - accuracy: 0.9435 - val_loss: 0.8892 - val_accuracy: 0.6184
Epoch 4/100
177/177 [==============================] - 0s 115us/step - loss: 0.2386 - accuracy: 0.9492 - val_loss: 0.8905 - val_accuracy: 0.6184
Epoch 5/100
177/177 [==============================] - 0s 98us/step - loss: 0.2358 - accuracy: 0.9379 - val_loss: 0.8889 - val_accuracy: 0.6184
Epoch 6/100
177/177 [==============================] - 0s 97us/step - loss: 0.2373 - accuracy: 0.9435 - val_loss: 0.8886 - val_accuracy: 0.6184
Epoch 7/100
177/177 [==============================] - 0s 101us/step - loss: 0.2317 - accu

In [240]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel2.history['accuracy'])*100))

train accuracy: 96.95%


In [84]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [85]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,CFBREBSa104,1,2,1.290039e-09,1.567630e-07,9.999999e-01
1,p0017kpresabs_qual,NRS199,2,2,2.856965e-05,2.843749e-03,9.971277e-01
2,p0017kpresabs_qual,NRS233,1,0,9.999446e-01,4.541289e-06,5.090974e-05
3,p0017kpresabs_qual,SR1746,0,2,6.282367e-02,7.075194e-04,9.364688e-01
4,p0017kpresabs_qual,NRS202,2,2,8.229589e-03,2.163908e-05,9.917488e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,GA27,2,1,3.964591e-03,9.959286e-01,1.068284e-04
604,p0040presabsSTCC_qual,GA231,2,1,3.309226e-04,9.996691e-01,6.232397e-10
605,p0040presabsSTCC_qual,SR1287,0,1,8.445997e-06,9.999915e-01,1.182947e-13
606,p0040presabsSTCC_qual,506,2,0,6.516150e-01,1.480882e-02,3.335762e-01


In [86]:
y_prob6 = df_proba6[df_proba6['phage']=='p0017presabs_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[2.22783210e-01, 3.92377700e-02, 7.37979050e-01],
       [1.90074920e-01, 2.06699490e-01, 6.03225600e-01],
       [5.00600460e-01, 2.06930040e-01, 2.92469470e-01],
       [3.50030750e-01, 1.68527270e-02, 6.33116540e-01],
       [9.61318500e-03, 4.35730600e-02, 9.46813760e-01],
       [1.96084900e-02, 3.02962480e-01, 6.77429100e-01],
       [5.31811030e-03, 2.47340170e-02, 9.69947900e-01],
       [6.63762000e-01, 2.74558340e-01, 6.16796580e-02],
       [2.40310800e-02, 9.45180240e-01, 3.07886770e-02],
       [5.51100500e-01, 3.27758400e-01, 1.21141030e-01],
       [2.69468930e-01, 5.62479850e-01, 1.68051230e-01],
       [2.92012730e-02, 6.46264300e-01, 3.24534420e-01],
       [6.47047940e-01, 3.19830360e-01, 3.31217800e-02],
       [1.46860880e-02, 4.71504480e-02, 9.38163400e-01],
       [6.98584260e-01, 2.82217030e-01, 1.91987300e-02],
       [1.21431640e-01, 4.56085000e-01, 4.22483350e-01],
       [4.79260620e-01, 3.67251800e-01, 1.53487560e-01],
       [7.86176150e-01, 1.16117

In [87]:
ovo6 = rocauc_ovo(y_sel_test, y_prob6, average="macro", multi_class="ovo")
ovo6

0.7905185643280882

In [88]:
ovr6 = rocauc_ovr(y_sel_test, y_prob6, average="macro", multi_class="ovr")
ovr6

0.7905185643280882

In [89]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_sel)

In [90]:
dat7 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat7['test'] = y_sel_test

In [91]:
dat7

,strain,test
222,NRS271,1
242,SR2852,2
34,CA39,2
221,NRS266,1
230,NY356,2
...,...,...
105,MN055,2
67,CFBRSa04,0
235,SR1065,0
190,NRS224,1


In [92]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [107]:
model_sel3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [108]:
model_sel3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [109]:
model_sel3.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 889us/step - loss: 1.0886 - accuracy: 0.4520 - val_loss: 1.0580 - val_accuracy: 0.4211
Epoch 2/100
177/177 [==============================] - 0s 342us/step - loss: 0.9605 - accuracy: 0.5424 - val_loss: 1.0466 - val_accuracy: 0.4868
Epoch 3/100
177/177 [==============================] - 0s 281us/step - loss: 0.9203 - accuracy: 0.5537 - val_loss: 1.0472 - val_accuracy: 0.4737
Epoch 4/100
177/177 [==============================] - 0s 269us/step - loss: 0.8863 - accuracy: 0.6045 - val_loss: 1.0410 - val_accuracy: 0.5395
Epoch 5/100
177/177 [==============================] - 0s 237us/step - loss: 0.8428 - accuracy: 0.6497 - val_loss: 1.0127 - val_accuracy: 0.4737
Epoch 6/100
177/177 [==============================] - 0s 297us/step - loss: 0.8386 - accuracy: 0.6610 - val_loss: 1.0035 - val_accuracy: 0.5000
Epoch 7/100
177/177 [==============================] - 0s 284us/step - loss: 0.7976 -

Epoch 57/100
177/177 [==============================] - 0s 340us/step - loss: 0.1618 - accuracy: 0.9435 - val_loss: 1.4755 - val_accuracy: 0.5132
Epoch 58/100
177/177 [==============================] - 0s 231us/step - loss: 0.1824 - accuracy: 0.9379 - val_loss: 1.4936 - val_accuracy: 0.5132
Epoch 59/100
177/177 [==============================] - 0s 371us/step - loss: 0.1665 - accuracy: 0.9548 - val_loss: 1.4787 - val_accuracy: 0.5263
Epoch 60/100
177/177 [==============================] - 0s 420us/step - loss: 0.1541 - accuracy: 0.9548 - val_loss: 1.4259 - val_accuracy: 0.5395
Epoch 61/100
177/177 [==============================] - 0s 278us/step - loss: 0.1432 - accuracy: 0.9661 - val_loss: 1.4613 - val_accuracy: 0.5395
Epoch 62/100
177/177 [==============================] - 0s 209us/step - loss: 0.1726 - accuracy: 0.9435 - val_loss: 1.4678 - val_accuracy: 0.5395
Epoch 63/100
177/177 [==============================] - 0s 321us/step - loss: 0.1333 - accuracy: 0.9605 - val_loss: 1.5058 -

In [280]:
acc_test_sel3 = model_sel3.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel3*100))

76/76 [==============================] - 0s 69us/step
test accuracy: 59.21%


In [110]:
pred7 = model_sel3.predict_classes(X_sel_test)
pred7

array([1, 2, 1, 0, 1, 0, 2, 2, 2, 0, 2, 1, 2, 0, 2, 1, 2, 1, 2, 2, 2, 1,
       1, 1, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 0, 2, 2, 2, 2, 0,
       1, 0, 1, 1, 0, 2, 1, 2, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 0, 2, 0, 2,
       1, 0, 1, 0, 0, 2, 0, 0, 2, 2])

In [111]:
dat7['pred'] = pred7
dat7

,strain,test,pred
222,NRS271,1,1
242,SR2852,2,2
34,CA39,2,1
221,NRS266,1,0
230,NY356,2,1
...,...,...,...
105,MN055,2,2
67,CFBRSa04,0,0
235,SR1065,0,0
190,NRS224,1,2


In [112]:
proba7 = model_sel3.predict_proba(X_sel_test)
dat_proba7 = pd.DataFrame(proba7)

In [113]:
dat_proba7

,0,1,2
0,0.103515,0.896367,0.000118
1,0.000002,0.000711,0.999287
2,0.021677,0.789370,0.188953
3,0.782633,0.007122,0.210245
4,0.041063,0.812126,0.146811
...,...,...,...
71,0.002503,0.000234,0.997263
72,0.997899,0.002089,0.000012
73,0.999928,0.000020,0.000052
74,0.155618,0.223443,0.620939


In [114]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [115]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p17p.csv", index = False,
         header=None)

In [284]:
hist_sel3 = model_sel3.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 197us/step - loss: 0.1839 - accuracy: 0.9379 - val_loss: 1.5561 - val_accuracy: 0.5789
Epoch 2/100
177/177 [==============================] - 0s 232us/step - loss: 0.1330 - accuracy: 0.9548 - val_loss: 1.4493 - val_accuracy: 0.5658
Epoch 3/100
177/177 [==============================] - 0s 268us/step - loss: 0.0947 - accuracy: 0.9718 - val_loss: 1.5021 - val_accuracy: 0.6053
Epoch 4/100
177/177 [==============================] - 0s 215us/step - loss: 0.1296 - accuracy: 0.9548 - val_loss: 1.5023 - val_accuracy: 0.6053
Epoch 5/100
177/177 [==============================] - 0s 196us/step - loss: 0.2210 - accuracy: 0.9209 - val_loss: 1.5517 - val_accuracy: 0.5921
Epoch 6/100
177/177 [==============================] - 0s 201us/step - loss: 0.1125 - accuracy: 0.9661 - val_loss: 1.6257 - val_accuracy: 0.6053
Epoch 7/100
177/177 [==============================] - 0s 293us/step - loss: 0.1927 -

Epoch 57/100
177/177 [==============================] - 0s 241us/step - loss: 0.0755 - accuracy: 0.9718 - val_loss: 1.6361 - val_accuracy: 0.5789
Epoch 58/100
177/177 [==============================] - 0s 208us/step - loss: 0.0627 - accuracy: 0.9831 - val_loss: 1.6318 - val_accuracy: 0.5921
Epoch 59/100
177/177 [==============================] - 0s 219us/step - loss: 0.0551 - accuracy: 0.9831 - val_loss: 1.6241 - val_accuracy: 0.5789
Epoch 60/100
177/177 [==============================] - 0s 272us/step - loss: 0.0543 - accuracy: 0.9831 - val_loss: 1.6179 - val_accuracy: 0.5789
Epoch 61/100
177/177 [==============================] - 0s 575us/step - loss: 0.0513 - accuracy: 0.9831 - val_loss: 1.6259 - val_accuracy: 0.5921
Epoch 62/100
177/177 [==============================] - 0s 381us/step - loss: 0.0674 - accuracy: 0.9774 - val_loss: 1.6594 - val_accuracy: 0.5921
Epoch 63/100
177/177 [==============================] - 0s 692us/step - loss: 0.0671 - accuracy: 0.9774 - val_loss: 1.6172 -

In [285]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel3.history['accuracy'])*100))

train accuracy: 97.03%


In [93]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [94]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS271,1,0,0.854929,0.144940,0.000130
1,p0017kpresabs_qual,SR2852,2,2,0.000001,0.000075,0.999923
2,p0017kpresabs_qual,CA39,2,0,0.959992,0.013406,0.026602
3,p0017kpresabs_qual,NRS266,1,2,0.000018,0.000003,0.999980
4,p0017kpresabs_qual,NY356,2,2,0.009156,0.411991,0.578852
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,CFBREBSa110,2,2,0.044135,0.010244,0.945621
604,p0040presabsSTCC_qual,CFBRSa05,0,2,0.145032,0.408530,0.446439
605,p0040presabsSTCC_qual,CFBREBSa123,0,0,0.994623,0.004152,0.001224
606,p0040presabsSTCC_qual,NY360,1,1,0.000795,0.948566,0.050639


In [95]:
y_prob7 = df_proba7[df_proba7['phage']=='p0017presabs_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[1.03515126e-01, 8.96367100e-01, 1.17821620e-04],
       [1.92032260e-06, 7.11404900e-04, 9.99286700e-01],
       [2.16771510e-02, 7.89370360e-01, 1.88952500e-01],
       [7.82633240e-01, 7.12198800e-03, 2.10244760e-01],
       [4.10627300e-02, 8.12126160e-01, 1.46811050e-01],
       [9.92544700e-01, 7.42333100e-03, 3.19968460e-05],
       [4.82616350e-02, 4.15503300e-02, 9.10188100e-01],
       [5.30144700e-03, 7.94538000e-02, 9.15244760e-01],
       [1.07479210e-02, 1.62570200e-01, 8.26681850e-01],
       [9.96360600e-01, 3.62630070e-03, 1.30336460e-05],
       [1.22501300e-01, 7.35098750e-07, 8.77498030e-01],
       [6.23069600e-04, 9.99066650e-01, 3.10273750e-04],
       [2.84500910e-05, 1.10161440e-03, 9.98869960e-01],
       [6.97613200e-01, 2.44939420e-03, 2.99937500e-01],
       [3.51189200e-01, 1.03502360e-01, 5.45308400e-01],
       [5.65149300e-02, 5.60856940e-01, 3.82628100e-01],
       [9.45280700e-02, 7.40330600e-03, 8.98068670e-01],
       [2.35872130e-01, 6.82258

In [96]:
ovo7 = rocauc_ovo(y_sel_test, y_prob7, average="macro", multi_class="ovo")
ovo7

0.7470624613481757

In [97]:
ovr7 = rocauc_ovr(y_sel_test, y_prob7, average="macro", multi_class="ovr")
ovr7

0.7470624613481757

In [98]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_sel)

In [99]:
dat8 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat8['test'] = y_sel_test

In [100]:
dat8

,strain,test
131,NRS104,0
123,NRS071,0
124,NRS072,1
27,BCH-SA-12,0
57,CFBREBSa125,2
...,...,...
170,NRS199,2
32,CA11,2
212,NRS253,1
141,NRS119,2


In [101]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [102]:
model_sel4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [121]:
model_sel4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [122]:
model_sel4.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 783us/step - loss: 1.1477 - accuracy: 0.4294 - val_loss: 1.0194 - val_accuracy: 0.5263
Epoch 2/100
177/177 [==============================] - 0s 223us/step - loss: 0.9896 - accuracy: 0.5424 - val_loss: 0.9884 - val_accuracy: 0.5395
Epoch 3/100
177/177 [==============================] - 0s 299us/step - loss: 0.9581 - accuracy: 0.5593 - val_loss: 1.0095 - val_accuracy: 0.5000
Epoch 4/100
177/177 [==============================] - 0s 237us/step - loss: 0.9183 - accuracy: 0.5876 - val_loss: 0.9838 - val_accuracy: 0.4868
Epoch 5/100
177/177 [==============================] - 0s 292us/step - loss: 0.8800 - accuracy: 0.6271 - val_loss: 0.9852 - val_accuracy: 0.5263
Epoch 6/100
177/177 [==============================] - 0s 223us/step - loss: 0.8550 - accuracy: 0.6215 - val_loss: 0.9867 - val_accuracy: 0.5263
Epoch 7/100
177/177 [==============================] - 0s 290us/step - loss: 0.8620 -

Epoch 57/100
177/177 [==============================] - 0s 153us/step - loss: 0.2918 - accuracy: 0.9153 - val_loss: 1.0445 - val_accuracy: 0.5789
Epoch 58/100
177/177 [==============================] - 0s 129us/step - loss: 0.2913 - accuracy: 0.9040 - val_loss: 1.0532 - val_accuracy: 0.5658
Epoch 59/100
177/177 [==============================] - 0s 130us/step - loss: 0.2976 - accuracy: 0.9096 - val_loss: 1.0494 - val_accuracy: 0.5526
Epoch 60/100
177/177 [==============================] - 0s 149us/step - loss: 0.3004 - accuracy: 0.8983 - val_loss: 1.0485 - val_accuracy: 0.5658
Epoch 61/100
177/177 [==============================] - 0s 143us/step - loss: 0.2757 - accuracy: 0.9266 - val_loss: 1.0701 - val_accuracy: 0.5658
Epoch 62/100
177/177 [==============================] - 0s 197us/step - loss: 0.3231 - accuracy: 0.9096 - val_loss: 1.1498 - val_accuracy: 0.5526
Epoch 63/100
177/177 [==============================] - 0s 319us/step - loss: 0.2967 - accuracy: 0.8927 - val_loss: 1.0881 -

In [304]:
acc_test_sel4 = model_sel4.evaluate(X_sel_test, y_sel_test)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_sel4*100))

76/76 [==============================] - 0s 128us/step
over-sampling test accuracy: 59.21%


In [123]:
pred8 = model_sel4.predict_classes(X_sel_test)
pred8

array([1, 2, 0, 1, 2, 1, 2, 2, 1, 0, 1, 2, 2, 0, 0, 2, 1, 1, 0, 1, 2, 2,
       2, 1, 0, 2, 2, 0, 2, 1, 2, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 0, 2,
       0, 2, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 1, 2, 2, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 2, 0])

In [124]:
dat8['pred'] = pred8
dat8

,strain,test,pred
131,NRS104,0,1
123,NRS071,0,2
124,NRS072,1,0
27,BCH-SA-12,0,1
57,CFBREBSa125,2,2
...,...,...,...
170,NRS199,2,0
32,CA11,2,0
212,NRS253,1,1
141,NRS119,2,2


In [125]:
proba8 = model_sel4.predict_proba(X_sel_test)
dat_proba8 = pd.DataFrame(proba8)

In [126]:
dat_proba8

,0,1,2
0,0.158166,0.745738,0.096096
1,0.038728,0.463862,0.497409
2,0.532130,0.070925,0.396945
3,0.198760,0.657271,0.143969
4,0.016703,0.248682,0.734616
...,...,...,...
71,0.456327,0.103251,0.440421
72,0.898749,0.093794,0.007457
73,0.044462,0.917985,0.037553
74,0.000014,0.055589,0.944397


In [127]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [128]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p17p.csv", index = False,
         header=None)

In [308]:
hist_sel4 = model_sel4.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 174us/step - loss: 0.1575 - accuracy: 0.9548 - val_loss: 1.2473 - val_accuracy: 0.5789
Epoch 2/100
177/177 [==============================] - 0s 233us/step - loss: 0.1468 - accuracy: 0.9718 - val_loss: 1.2577 - val_accuracy: 0.5658
Epoch 3/100
177/177 [==============================] - 0s 225us/step - loss: 0.1457 - accuracy: 0.9718 - val_loss: 1.2536 - val_accuracy: 0.5789
Epoch 4/100
177/177 [==============================] - 0s 228us/step - loss: 0.1483 - accuracy: 0.9718 - val_loss: 1.2536 - val_accuracy: 0.5658
Epoch 5/100
177/177 [==============================] - 0s 169us/step - loss: 0.1472 - accuracy: 0.9605 - val_loss: 1.2810 - val_accuracy: 0.5789
Epoch 6/100
177/177 [==============================] - 0s 186us/step - loss: 0.1456 - accuracy: 0.9605 - val_loss: 1.3070 - val_accuracy: 0.5658
Epoch 7/100
177/177 [==============================] - 0s 208us/step - loss: 0.1374 -

Epoch 57/100
177/177 [==============================] - 0s 169us/step - loss: 0.0833 - accuracy: 0.9831 - val_loss: 1.5934 - val_accuracy: 0.5789
Epoch 58/100
177/177 [==============================] - 0s 166us/step - loss: 0.0839 - accuracy: 0.9887 - val_loss: 1.6311 - val_accuracy: 0.5658
Epoch 59/100
177/177 [==============================] - 0s 196us/step - loss: 0.1145 - accuracy: 0.9492 - val_loss: 1.6507 - val_accuracy: 0.5658
Epoch 60/100
177/177 [==============================] - 0s 235us/step - loss: 0.0964 - accuracy: 0.9661 - val_loss: 1.6470 - val_accuracy: 0.5658
Epoch 61/100
177/177 [==============================] - 0s 221us/step - loss: 0.0902 - accuracy: 0.9718 - val_loss: 1.6329 - val_accuracy: 0.5658
Epoch 62/100
177/177 [==============================] - 0s 187us/step - loss: 0.1687 - accuracy: 0.9435 - val_loss: 1.7520 - val_accuracy: 0.5658
Epoch 63/100
177/177 [==============================] - 0s 169us/step - loss: 0.1263 - accuracy: 0.9492 - val_loss: 1.6782 -

In [309]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel4.history['accuracy'])*100))

train accuracy: 97.43%


In [103]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [104]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS104,0,1,8.210638e-02,0.917809,8.477923e-05
1,p0017kpresabs_qual,NRS071,0,2,1.015229e-02,0.255327,7.345203e-01
2,p0017kpresabs_qual,NRS072,1,2,3.642946e-03,0.000002,9.963547e-01
3,p0017kpresabs_qual,BCH-SA-12,0,2,2.809318e-03,0.099188,8.980029e-01
4,p0017kpresabs_qual,CFBREBSa125,2,0,4.875667e-01,0.088657,4.237761e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS113,1,2,2.686909e-07,0.093121,9.068785e-01
604,p0040presabsSTCC_qual,BCH-SA-09,2,1,5.529492e-07,0.999990,9.559324e-06
605,p0040presabsSTCC_qual,NRS106,2,1,1.115902e-03,0.998816,6.852559e-05
606,p0040presabsSTCC_qual,CFBREBSa131,2,2,2.708040e-07,0.156768,8.432316e-01


In [105]:
y_prob8 = df_proba8[df_proba8['phage']=='p0017presabs_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[1.58166140e-01, 7.45737970e-01, 9.60958900e-02],
       [3.87284900e-02, 4.63862100e-01, 4.97409500e-01],
       [5.32129760e-01, 7.09250000e-02, 3.96945240e-01],
       [1.98760090e-01, 6.57270800e-01, 1.43969130e-01],
       [1.67028960e-02, 2.48681580e-01, 7.34615500e-01],
       [2.38739430e-01, 7.27275130e-01, 3.39854250e-02],
       [3.66635770e-02, 4.03542760e-01, 5.59793600e-01],
       [3.05971830e-03, 2.49188860e-02, 9.72021400e-01],
       [3.77392720e-03, 5.35890100e-01, 4.60335970e-01],
       [9.57234740e-01, 3.91080400e-02, 3.65726980e-03],
       [1.87064260e-01, 5.71198640e-01, 2.41737040e-01],
       [3.39960870e-02, 1.55446290e-01, 8.10557660e-01],
       [1.22354170e-03, 1.91438700e-01, 8.07337760e-01],
       [9.64208900e-01, 2.55108760e-02, 1.02802210e-02],
       [7.69508500e-01, 1.56375450e-01, 7.41160140e-02],
       [8.57975200e-03, 1.51484090e-01, 8.39936200e-01],
       [8.06468200e-04, 9.16705200e-01, 8.24883000e-02],
       [1.99934160e-03, 7.71918

In [106]:
ovo8 = rocauc_ovo(y_sel_test, y_prob8, average="macro", multi_class="ovo")
ovo8

0.7692743764172336

In [107]:
ovr8 = rocauc_ovr(y_sel_test, y_prob8, average="macro", multi_class="ovr")
ovr8

0.7692743764172336

In [108]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.7719320472296664

In [109]:
np.std(ovos2)

0.016209217735817724

In [110]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.7719320472296664

In [111]:
np.std(ovrs2)

0.016209217735817724

In [310]:
accs_l= [acc_test_sel, acc_test_sel2, acc_test_sel3, acc_test_sel4]

In [311]:
mean_l = np.mean(accs_l)
print('test accuracy mean after lasso: %.2f%%' % (mean_l*100))

test accuracy mean after lasso: 60.86%


In [312]:
std_l = np.std(accs_l)
print('test accuracy standard deviation after lasso:', std_l)

test accuracy standard deviation after lasso: 0.017092598805124246


In [313]:
accs_train_l = [np.mean(hist_sel.history['accuracy']), np.mean(hist_sel2.history['accuracy']), np.mean(hist_sel3.history['accuracy']),
             np.mean(hist_sel4.history['accuracy'])]

In [314]:
mean_train_l = np.mean(accs_train_l)
print('train accuracy mean after lasso: %.2f%%' % (mean_train_l*100))

train accuracy mean after lasso: 96.97%


In [315]:
std_train_l = np.std(accs_train_l)
print('train accuracy standard deviation after lasso:', std_train_l)

train accuracy standard deviation after lasso: 0.0034746933
